In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available 1


In [4]:
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives, BinaryAccuracy, Precision, Recall, AUC
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Activation, SpatialDropout2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from typing import Tuple
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

In [ ]:
from preprocessing import train_val_test_split, parse_input_file

In [174]:
# directory to image folder - change this accordingly
DTIF = os.path.join('data', 'img')
DF_FILEPATH = 'list_category_img.txt'

In [175]:
df = parse_input_file(DF_FILEPATH, DTIF)
train_df, val_df, test_df = train_val_test_split(df)

In [8]:
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [13]:
train_gen = train_datagen.flow_from_dataframe(
    cat_df,
    weight_col=None, target_size=(301, 301), color_mode='rgb',
    classes=None, 
    batch_size=8,
    class_mode='categorical', 
    shuffle=True
)

Found 24362 validated image filenames belonging to 26 classes.


In [10]:
def base_model() -> Sequential:
    model = Sequential()
    model.add(Input((301, 301, 3)))    
    # Conv2D Layer
    model.add(Conv2D(filters=64, kernel_size=5, padding='same'))
    model.add(Activation('relu'))    
    model.add(SpatialDropout2D(0.5))

    # BatchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))    

    # Max Pooling
    model.add(MaxPooling2D(pool_size=(5, 5), strides = 5))

    model.add(Flatten())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.4))

    model.add(Dense(26, activation='softmax'))
    
    return model  

In [11]:
METRICS = [
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      AUC(name='prc', curve='PR')
]

model = base_model()
model.compile(optimizer='adam', loss=CategoricalCrossentropy(), metrics=METRICS)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 301, 301, 64)      4864      
_________________________________________________________________
activation (Activation)      (None, 301, 301, 64)      0         
_________________________________________________________________
spatial_dropout2d (SpatialDr (None, 301, 301, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 301, 301, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 301, 301, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 60, 60, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 230400)            0

In [14]:
epochs=10
history = model.fit(
    train_gen,
    epochs=epochs
)

Epoch 1/10
3046/3046 [==============================] - 376s 123ms/step - loss: 2.7648 - accuracy: 0.9615
Epoch 2/10
3046/3046 [==============================] - 284s 93ms/step - loss: 2.5133 - accuracy: 0.9615s - loss: 2.5
Epoch 3/10
3046/3046 [==============================] - 295s 97ms/step - loss: 2.5125 - accuracy: 0.9615
Epoch 4/10
3046/3046 [==============================] - 295s 97ms/step - loss: 2.5007 - accuracy: 0.9615
Epoch 5/10
3046/3046 [==============================] - 295s 97ms/step - loss: 2.5141 - accuracy: 0.9615
Epoch 6/10
3046/3046 [==============================] - 295s 97ms/step - loss: 2.5086 - accuracy: 0.9615
Epoch 7/10
3046/3046 [==============================] - 295s 97ms/step - loss: 2.5013 - accuracy: 0.9615
Epoch 8/10
3046/3046 [==============================] - 295s 97ms/step - loss: 2.4999 - accuracy: 0.9615
Epoch 9/10
3046/3046 [==============================] - 295s 97ms/step - loss: 2.5000 - accuracy: 0.9615
Epoch 10/10
3046/3046 [==================

In [29]:
from tensorflow.keras.callbacks import History

In [28]:
type(history)

keras.callbacks.History

In [ ]:
def compute_metrics():
    pass

In [19]:
def plot_metrics(history):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    metrics = ['accuracy']
    fig, ax = plt.subplots(2, 2, figsize=(20,8))
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        if metric == 'loss':
          pos = (0, 0)
        elif metric == 'accuracy':
          pos = (0, 1)
        elif metric == 'precision':
          pos = (1, 0)
        elif metric == 'recall':
          pos = (1, 1)
        ax[pos[0], pos[1]].plot(history.epoch, history.history[metric], color=colors[0], label='Train')
        ax[pos[0], pos[1]].plot(history.epoch, history.history['val_'+metric],
                color=colors[0], linestyle="--", label='Val')
        ax[pos[0], pos[1]].set_xlabel('Epoch')
        ax[pos[0], pos[1]].set_ylabel(name)
        ax[pos[0], pos[1]].set_ylim([0,1])

    ax[pos[0], pos[1]].legend()

def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = roc_curve(labels, predictions)
    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    plt.xlim([-0.5,20])
    plt.ylim([80,100.5])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')